In [1]:
from __future__ import division
import pandas as pd
import numpy as np
from scipy.constants import codata

In [5]:
path=r"C:\Users\cjang\Desktop\impedance_data\data\\"
data = ['DE_40_2_60.mpt']

In [6]:

def push(path, data, cycle='off', mask=['none','none']):
    df_raw0 = []
    cycleno = []
    for j in range(len(data)):
        if data[j].find(".mpt") != -1: #file is a .mpt file
            df_raw0.append(extract_mpt(path=path, EIS_name=data[j])) #reads all datafiles
        elif data[j].find(".DTA") != -1: #file is a .dta file
            df_raw0.append(extract_dta(path=path, EIS_name=data[j])) #reads all datafiles
        elif data[j].find(".z") != -1: #file is a .z file
            df_raw0.append(extract_solar(path=path, EIS_name=data[j])) #reads all datafiles
        else:
            print('Data file(s) could not be identified')

        cycleno.append(df_raw0[j].cycle_number)
        if np.min(cycleno[j]) <= np.max(cycleno[j-1]):
            if j > 0: #corrects cycle_number except for the first data file
                df_raw0[j].update({'cycle_number': cycleno[j]+np.max(cycleno[j-1])}) #corrects cycle number

    df_raw = [i for i in df_raw0][0]
    return df_raw



#TAKEN FROM PYEIS LIBRARY
def correct_text_EIS(text_header):
    '''Corrects the text of '*.mpt' and '*.dta' files into readable parameters without spaces, ., or /
    
    <E_we> = averaged Wew value for each frequency
    <I> = Averaged I values for each frequency
    |E_we| = module of Ewe
    |I_we| = module of Iwe
    Cs/F = Capacitance caluculated using an R+C (series) equivalent circuit
    Cp/F = Capacitance caluculated using an R-C (parallel) equivalent circuit
    Ref.:
        - EC-Lab User's Manual
    
    Kristian B. Knudsen (kknu@berkeley.edu || kristianbknudsen@gmail.com)
    '''
    if text_header == 'freq/Hz' or text_header == '  Freq(Hz)':
        return 'f'
    elif text_header == 'Re(Z)/Ohm' or text_header == "Z'(a)":
        return 're'
    elif text_header == '-Im(Z)/Ohm' or text_header == "Z''(b)":
        return 'im'
#    elif text_header == "Z''(b)":
#        return 'im_neg'
    elif text_header == '|Z|/Ohm':
        return 'Z_mag'
    elif text_header == 'Phase(Z)/deg':
        return 'Z_phase'
    elif text_header == 'time/s' or text_header == 'Time(Sec)':
        return 'times'
    elif text_header == '<Ewe>/V' or text_header == 'Bias':
        return 'E_avg'
    elif text_header == '<I>/mA':
        return 'I_avg'
    elif text_header == 'Cs/F':
        return 'Cs' ####
    elif text_header == 'Cp/F':
        return 'Cp'
    elif text_header == 'cycle number':
        return 'cycle_number'
    elif text_header == 'Re(Y)/Ohm-1':
        return 'Y_re'
    elif text_header == 'Im(Y)/Ohm-1':
        return 'Y_im'
    elif text_header == '|Y|/Ohm-1':
        return 'Y_mag'
    elif text_header == 'Phase(Y)/deg':
        return 'Y_phase'
    elif text_header == 'Time':
        return 'times'
    elif text_header == 'Freq':
        return 'f'
    elif text_header == 'Zreal':
        return 're'
    elif text_header == 'Zimag':
        return 'im'
    elif text_header == 'Zmod':
        return 'Z_mag'
    elif text_header == 'Vdc':
        return 'E_avg'
    elif text_header == 'Idc':
        return 'I_avg'
    elif text_header == 'I/mA':
        return 'ImA'
    elif text_header == 'Ewe/V':
        return 'EweV'
    elif text_header == 'half cycle':
        return 'half_cycle'
    elif text_header == 'Ns changes':
        return 'Ns_changes'
    else:
        return text_header


#TAKEN FROM PYEIS LIBRARY
def extract_mpt(path, EIS_name):
    '''
    Extracting PEIS and GEIS data files from EC-lab '.mpt' format, coloums are renames following correct_text_EIS()
    
    Kristian B. Knudsen (kknu@berkeley.edu || kristianbknudsen@gmail.com)
    '''
    EIS_init = pd.read_csv(path+EIS_name, sep='\t', nrows=1,header=0,names=['err'], encoding='latin1') #findes line that states skiplines
#    EIS_test_header_names = pd.read_csv(path+EIS_name, sep='\t', skiprows=int(EIS_init.err[0][18:20])-1, encoding='latin1') #locates number of skiplines
    EIS_test_header_names = pd.read_csv(path+EIS_name, sep='\t', skiprows=int(EIS_init.err[0][18:-1])-1, encoding='latin1') #locates number of skiplines
    names_EIS = []
    for j in range(len(EIS_test_header_names.columns)):
        names_EIS.append(correct_text_EIS(EIS_test_header_names.columns[j])) #reads coloumn text
#    return pd.read_csv(path+EIS_name, sep='\t', skiprows=int(EIS_init.err[0][18:20]), names=names_EIS, encoding='latin1')
    return pd.read_csv(path+EIS_name, sep='\t', skiprows=int(EIS_init.err[0][18:-1]), names=names_EIS, encoding='latin1')


#TAKEN FROM PYEIS LIBRARY
def extract_dta(path, EIS_name):
    '''
    Extracting data files from Gamry '.DTA' format, coloums are renames following correct_text_EIS()
    
    Kristian B. Knudsen (kknu@berkeley.edu || kristianbknudsen@gmail.com)
    '''
    dummy_col = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I','J','K','L','M','N','O','P']
    init = pd.read_csv(path+EIS_name, encoding='latin1', sep='\t', names=dummy_col)
    ZC = pd.Index(init.A)
    header_loc = ZC.get_loc('ZCURVE')+1  ##ZC.get_loc('ZCURVE')+3
    
    header_names_raw = pd.read_csv(path+EIS_name, sep='\t', skiprows=header_loc, encoding='latin1') #locates number of skiplines
    header_names = []
    for j in range(len(header_names_raw.columns)):
        header_names.append(correct_text_EIS(header_names_raw.columns[j])) #reads coloumn text
    data = pd.read_csv(path+EIS_name, sep='\t', skiprows=ZC.get_loc('ZCURVE')+3, names=header_names, encoding='latin1')
    data.update({'im': np.abs(data.im)})
    data = data.assign(cycle_number = 1.0)
    return data


#TAKEN FROM PYEIS LIBRARY
def extract_solar(path, EIS_name):
    '''
    Extracting data files from Solartron's '.z' format, coloums are renames following correct_text_EIS()
    
    Kristian B. Knudsen (kknu@berkeley.edu || kristianbknudsen@gmail.com)
    '''
    dummy_col = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I','J','K','L','M','N','O','P']
    init = pd.read_csv(path+EIS_name, encoding='latin1', sep='\t', names=dummy_col)
    ZC = pd.Index(init.A)
    header_loc = ZC.get_loc('  Freq(Hz)')
    
    header_names_raw = pd.read_csv(path+EIS_name, sep='\t', skiprows=header_loc, encoding='latin1') #locates number of skiplines
    header_names = []
    for j in range(len(header_names_raw.columns)):
        header_names.append(correct_text_EIS(header_names_raw.columns[j])) #reads coloumn text
    data = pd.read_csv(path+EIS_name, sep='\t', skiprows=header_loc+2, names=header_names, encoding='latin1')
    data.update({'im': -data.im})
    data = data.assign(cycle_number = 1.0)
    return data

#
#print()
#print('---> Data Extraction Script Loaded (v. 0.0.2 - 06/27/18)')

In [7]:
push(path,data)

,f,re,im,Z_mag,Z_phase,times,E_avg,I_avg,Cs/µF,Cp/µF,...,|I|/A,(Q-Qo)/mA.h,Analog OUT/V,Analog IN 1/V,Y_re,Y_im,Y_mag,Y_phase,dq/mA.h,Unnamed: 22
0,1.000019e+06,54.512085,12.467592,55.919659,-12.882680,1.644481,-0.035080,1.426269e-03,0.012765,0.000635,...,0.000017,3.447100e-07,0.0,-100.10986,0.017433,0.003987,0.017883,12.882680,3.447100e-07,NaN
1,8.912467e+05,54.781143,9.948639,55.677185,-10.293134,2.240487,-0.034646,1.666198e-03,0.017950,0.000573,...,0.000028,7.131817e-07,0.0,-100.11788,0.017672,0.003209,0.017961,10.293134,3.684718e-07,NaN
2,7.943327e+05,55.901787,8.067935,56.480984,-8.212415,2.836475,-0.034303,1.556177e-03,0.024834,0.000507,...,0.000045,1.085397e-06,0.0,-100.09383,0.017524,0.002529,0.017705,8.212415,3.722156e-07,NaN
3,7.079498e+05,57.193165,6.863119,57.603477,-6.842713,3.432488,-0.034021,1.485742e-03,0.032756,0.000465,...,0.000068,1.277489e-06,0.0,-100.06977,0.017236,0.002068,0.017360,6.842713,1.920919e-07,NaN
4,6.309568e+05,58.379940,6.027622,58.690285,-5.894797,4.028492,-0.033781,1.421811e-03,0.041848,0.000441,...,0.000098,1.444308e-06,0.0,-100.10986,0.016948,0.001750,0.017039,5.894797,1.668191e-07,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,1.584685e+00,7832.707500,47199.789000,47845.285000,-80.577751,88.893994,-0.032893,-4.681060e-06,2.127830,2.070803,...,0.000003,1.034084e-05,0.0,-100.05373,0.000003,0.000021,0.000021,80.577751,-1.292154e-07,NaN
117,1.412238e+00,8873.412100,52384.191000,53130.414000,-80.385872,91.423164,-0.032897,7.205869e-06,2.151354,2.091347,...,0.000002,1.042559e-05,0.0,-100.07779,0.000003,0.000019,0.000019,80.385872,8.474895e-08,NaN
118,1.259674e+00,9995.331100,58012.574000,58867.355000,-80.224152,94.251954,-0.032894,-1.805065e-06,2.177909,2.115120,...,0.000002,1.040201e-05,0.0,-100.10184,0.000003,0.000017,0.000017,80.224152,-2.358587e-08,NaN
119,1.122486e+00,11278.340000,64216.504000,65199.391000,-80.038742,97.419985,-0.032894,4.672643e-06,2.207968,2.141899,...,0.000002,1.037314e-05,0.0,-100.09383,0.000003,0.000015,0.000015,80.038742,-2.887142e-08,NaN


In [ ]:
def correct_text_EIS(text_header):
    '''Corrects the text of '*.mpt' and '*.dta' files into readable parameters without spaces, ., or /
    
    <E_we> = averaged Wew value for each frequency
    <I> = Averaged I values for each frequency
    |E_we| = module of Ewe
    |I_we| = module of Iwe
    Cs/F = Capacitance caluculated using an R+C (series) equivalent circuit
    Cp/F = Capacitance caluculated using an R-C (parallel) equivalent circuit
    Ref.:
        - EC-Lab User's Manual
    
    Kristian B. Knudsen (kknu@berkeley.edu || kristianbknudsen@gmail.com)
    '''
    if text_header == 'freq/Hz' or text_header == '  Freq(Hz)':
        return 'f'
    elif text_header == 'Re(Z)/Ohm' or text_header == "Z'(a)":
        return 're'
    elif text_header == '-Im(Z)/Ohm' or text_header == "Z''(b)":
        return 'im'
#    elif text_header == "Z''(b)":
#        return 'im_neg'
    elif text_header == '|Z|/Ohm':
        return 'Z_mag'
    elif text_header == 'Phase(Z)/deg':
        return 'Z_phase'
    elif text_header == 'time/s' or text_header == 'Time(Sec)':
        return 'times'
    elif text_header == '<Ewe>/V' or text_header == 'Bias':
        return 'E_avg'
    elif text_header == '<I>/mA':
        return 'I_avg'
    elif text_header == 'Cs/F':
        return 'Cs' ####
    elif text_header == 'Cp/F':
        return 'Cp'
    elif text_header == 'cycle number':
        return 'cycle_number'
    elif text_header == 'Re(Y)/Ohm-1':
        return 'Y_re'
    elif text_header == 'Im(Y)/Ohm-1':
        return 'Y_im'
    elif text_header == '|Y|/Ohm-1':
        return 'Y_mag'
    elif text_header == 'Phase(Y)/deg':
        return 'Y_phase'
    elif text_header == 'Time':
        return 'times'
    elif text_header == 'Freq':
        return 'f'
    elif text_header == 'Zreal':
        return 're'
    elif text_header == 'Zimag':
        return 'im'
    elif text_header == 'Zmod':
        return 'Z_mag'
    elif text_header == 'Vdc':
        return 'E_avg'
    elif text_header == 'Idc':
        return 'I_avg'
    elif text_header == 'I/mA':
        return 'ImA'
    elif text_header == 'Ewe/V':
        return 'EweV'
    elif text_header == 'half cycle':
        return 'half_cycle'
    elif text_header == 'Ns changes':
        return 'Ns_changes'
    else:
        return text_header
    
def extract_mpt(path, EIS_name):
    '''
    Extracting PEIS and GEIS data files from EC-lab '.mpt' format, coloums are renames following correct_text_EIS()
    
    Kristian B. Knudsen (kknu@berkeley.edu || kristianbknudsen@gmail.com)
    '''
    EIS_init = pd.read_csv(path+EIS_name, sep='\t', nrows=1,header=0,names=['err'], encoding='latin1') #findes line that states skiplines
#    EIS_test_header_names = pd.read_csv(path+EIS_name, sep='\t', skiprows=int(EIS_init.err[0][18:20])-1, encoding='latin1') #locates number of skiplines
    EIS_test_header_names = pd.read_csv(path+EIS_name, sep='\t', skiprows=int(EIS_init.err[0][18:-1])-1, encoding='latin1') #locates number of skiplines
    names_EIS = []
    for j in range(len(EIS_test_header_names.columns)):
        names_EIS.append(correct_text_EIS(EIS_test_header_names.columns[j])) #reads coloumn text
#    return pd.read_csv(path+EIS_name, sep='\t', skiprows=int(EIS_init.err[0][18:20]), names=names_EIS, encoding='latin1')
    return pd.read_csv(path+EIS_name, sep='\t', skiprows=int(EIS_init.err[0][18:-1]), names=names_EIS, encoding='latin1')

def extract_dta(path, EIS_name):
    '''
    Extracting data files from Gamry '.DTA' format, coloums are renames following correct_text_EIS()
    
    Kristian B. Knudsen (kknu@berkeley.edu || kristianbknudsen@gmail.com)
    '''
    dummy_col = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I','J','K','L','M','N','O','P']
    init = pd.read_csv(path+EIS_name, encoding='latin1', sep='\t', names=dummy_col)
    ZC = pd.Index(init.A)
    header_loc = ZC.get_loc('ZCURVE')+1  ##ZC.get_loc('ZCURVE')+3
    
    header_names_raw = pd.read_csv(path+EIS_name, sep='\t', skiprows=header_loc, encoding='latin1') #locates number of skiplines
    header_names = []
    for j in range(len(header_names_raw.columns)):
        header_names.append(correct_text_EIS(header_names_raw.columns[j])) #reads coloumn text
    data = pd.read_csv(path+EIS_name, sep='\t', skiprows=ZC.get_loc('ZCURVE')+3, names=header_names, encoding='latin1')
    data.update({'im': np.abs(data.im)})
    data = data.assign(cycle_number = 1.0)
    return data

def extract_solar(path, EIS_name):
    '''
    Extracting data files from Solartron's '.z' format, coloums are renames following correct_text_EIS()
    
    Kristian B. Knudsen (kknu@berkeley.edu || kristianbknudsen@gmail.com)
    '''
    dummy_col = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I','J','K','L','M','N','O','P']
    init = pd.read_csv(path+EIS_name, encoding='latin1', sep='\t', names=dummy_col)
    ZC = pd.Index(init.A)
    header_loc = ZC.get_loc('  Freq(Hz)')
    
    header_names_raw = pd.read_csv(path+EIS_name, sep='\t', skiprows=header_loc, encoding='latin1') #locates number of skiplines
    header_names = []
    for j in range(len(header_names_raw.columns)):
        header_names.append(correct_text_EIS(header_names_raw.columns[j])) #reads coloumn text
    data = pd.read_csv(path+EIS_name, sep='\t', skiprows=header_loc+2, names=header_names, encoding='latin1')
    data.update({'im': -data.im})
    data = data.assign(cycle_number = 1.0)
    return data

#
#print()
#print('---> Data Extraction Script Loaded (v. 0.0.2 - 06/27/18)')